# Laboratory practice № 9 (Linear Matrix Inequalities)

Done by Vladislav Diuzhev (isu: 336225, group: R33353)

Variant: 8

## Preparation

In [100]:
import matplotlib.pyplot as plt
import control as ctrl
import sympy
import numpy as np
from sympy.utilities.lambdify import lambdify
import scipy
import cvxpy as cp

## Task 1

In [101]:
A = np.array([
    [-1, 0, 0, 0],
    [0, 2, 0, 0],
    [0, 0, 3, 4],
    [0, 0, -4, 3]
])

B = np.array([
    [0],
    [5],
    [0],
    [6]
])

In [102]:
np.linalg.eigvals(A)

array([ 3.+4.j,  3.-4.j, -1.+0.j,  2.+0.j])

In [103]:
alphas = [0.1, 0.5, 1]

In [156]:
def get_K_LMI(a, b, alpha):
    P = cp.Variable((4,4),PSD=True)
    Y = cp.Variable((1,4)) 
    prob = cp.Problem(cp.Maximize(0),[P >> np.eye(4), P@a.T + a@P + 2*alpha*P + Y.T @ b.T + b@Y << 0])
    prob.solve()
    print(P.value, Y.value)
    return Y.value @ np.linalg.inv(P.value) 

def get_K_LMI_mu(a, b, alpha, x0, mu=None):
    P = cp.Variable((4,4),PSD=True)
    Y = cp.Variable((1,4)) 
    if mu is None:
        mu_ = cp.Variable((1,1))
    else:
        mu_ = mu
    sub1 = cp.bmat([
        [P, x0],
        [x0.T, [[1]]]
    ])
    if mu is None:
        sub2 = cp.bmat([
            [P, Y.T],
            [Y, mu_]
        ])
    else:
        sub2 = cp.bmat([
            [P, Y.T],
            [Y, [[mu_*mu_]]]
        ])
    res = prob = cp.Problem(cp.Maximize(0) if mu is not None else cp.Minimize(mu_),
                      [P >> np.eye(4), 
                       P@a.T + a@P + 2*alpha*P + Y.T @ b.T + b@Y << 0,
                       sub1 >> 0, sub2 >> 0])
    prob.solve()
    print(P.value, Y.value)
    return Y.value @ np.linalg.inv(P.value), np.sqrt(res.value)

In [157]:
Ks = [get_K_LMI(A,B,alph) for alph in alphas]

[[  658.62362884     0.             0.             0.        ]
 [    0.          2559.9522096  -1244.63244745  1707.71045091]
 [    0.         -1244.63244745   810.07511982  -640.26490679]
 [    0.          1707.71045091  -640.26490679  1947.93990446]] [[    0.         -1089.1784326    -86.63386372 -1441.37987897]]
[[  872.45278574     0.             0.             0.        ]
 [    0.          3494.67399112 -1640.94180368  2552.06683463]
 [    0.         -1640.94180368  1031.84283688  -913.98437867]
 [    0.          2552.06683463  -913.98437867  2778.25857689]] [[    0.         -1761.40094083   -87.53633521 -2238.11261583]]
[[ 1246.7448991      0.             0.             0.        ]
 [    0.          4983.64016877 -2220.33379323  3973.97683045]
 [    0.         -2220.33379323  1347.0275877  -1356.23602621]
 [    0.          3973.97683045 -1356.23602621  4201.57703627]] [[    0.         -3003.65289183   -84.8805141  -3713.00314942]]


In [158]:
np.round(np.linalg.eigvals(A + B@Ks[0]),3)

array([-0.47+6.893j, -0.47-6.893j, -0.59+0.j   , -1.  +0.j   ])

In [159]:
np.round(np.linalg.eigvals(A + B@Ks[1]),3)

array([-0.884+7.444j, -0.884-7.444j, -0.983+0.j   , -1.   +0.j   ])

In [160]:
np.round(np.linalg.eigvals(A + B@Ks[2]),3)

array([-1.378+8.141j, -1.378-8.141j, -1.455+0.j   , -1.   +0.j   ])

## Task 2

In [161]:
X_0 = np.array([
    [1],
    [1],
    [1],
    [1]
])

In [165]:
K, res = get_K_LMI_mu(A,B,0.1,X_0)

[[309.19798418   5.72350992  -1.73685813   1.09457941]
 [  5.72350992  32.77819816 -16.13744388  17.99377376]
 [ -1.73685813 -16.13744388  10.2515925   -7.94437157]
 [  1.09457941  17.99377376  -7.94437157  18.98205385]] [[ -1.79501966 -13.76738712   2.38899421 -15.10387183]]


In [166]:
res

4.240468209294919

## Task 3

In [167]:
A = np.array([
    [0, 3, 0 ,0],
    [-3, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, -1, 0]
])
C = np.array([2, 0, 0, 3]).reshape((1,-1))

In [170]:
def get_L_LMI(a, c, alpha):
    Q = cp.Variable((4,4),PSD=True)
    Y = cp.Variable((4,1)) 
    prob = cp.Problem(cp.Maximize(0),[Q >> np.eye(4), 
                                      a.T@Q + Q@A + 2*alpha*Q + c.T@Y.T + Y@c << 0])
    prob.solve()
    print(Q.value, Y.value)
    return np.linalg.inv(Q.value) @ Y.value


In [183]:
get_L_LMI(A,C,9)

[[ 4.85548884e+09 -7.68053370e+08  3.91824616e+08  7.44034924e+09]
 [-7.68053370e+08  2.56016127e+08 -1.33909145e+08 -1.23482950e+09]
 [ 3.91824616e+08 -1.33909145e+08  7.02628960e+07  6.32428831e+08]
 [ 7.44034924e+09 -1.23482950e+09  6.32428831e+08  1.14326057e+10]] [[-2.30017809e+10]
 [ 1.32663361e+07]
 [-7.12085103e+06]
 [-3.45086270e+10]]


array([[1404.68090962],
       [2235.74544504],
       [4998.15624929],
       [-952.19227759]])